# Optimal window size as a function of the image and noise level

In [ ]:
import time
from collections import namedtuple

try:
    import numpy as np
except:
    !pip install numpy
    import numpy as np

import scipy.ndimage

try:
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.ticker as mticker
except:
    !pip install matplotlib
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.ticker as mticker

#from ipywidgets import *
try:
    import cv2
except:
    !pip install cv2
    import cv2
    
#import kernels
try:
    from skimage import io as skimage_io
except:
    !pip install scikit-image
    from skimage import io as skimage_io

try:
    import information_theory as IT
except:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
    import information_theory as IT

import utils

In [ ]:
# apt install cm-super-minimal
# apt install dvipng
plt.rcParams.update({
    "text.usetex": True,
    #"font.family": "Helvetica",
    "font.family": "Serif",
    "text.latex.preamble": r"\usepackage{amsmath} \usepackage{amsfonts}"
})

In [ ]:
import logging
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.WARNING)

In [ ]:
try:
    from scipy.signal import wiener
except:
    !pip install scipy
    from scipy.signal import wiener

### Confocal_FISH

In [ ]:
Args = namedtuple("args", ["clean"])
args = Args("http://www.hpca.ual.es/~vruiz/images/FDM/Confocal_FISH.png")

In [ ]:
X = skimage_io.imread(args.clean)

### Confocal_FISH_GF_optimal_tau

In [ ]:
sizes = [i for i in range(3, 15)] # Number of points per line
std_devs = [i for i in range(0, 200, 40)]
PCC_curves = []
poisson_ratio = 0.5
gamma = 0.15
for std_dev in std_devs: # Number of noise levels
    PCC_curve = []
    Y = utils.generate_MPGN(X, std_dev, gamma, poisson_ratio).reshape(X.shape)
    for i in sizes: # Window size
        denoised = wiener(Y.astype(np.float32), mysize=i)
        PCC = np.corrcoef(denoised.flatten(), X.flatten())[0, 1]
        print("std_dev:", std_dev, "size:", i, "PCC:", PCC)
        PCC_curve.append(PCC)

    PCC_curves.append(PCC_curve)
    print()

In [ ]:
optimal_sizes = []
i = 0
for std_dev in std_devs:
    optimal_size = sizes[np.argmax(PCC_curves[i])]
    optimal_sizes.append(optimal_size)
    plt.plot(sizes,
        PCC_curves[i],
        label=rf"$\sigma={std_dev}, \gamma={gamma}"
        + r", w^*="
        + rf"{optimal_size:.2f}"
        + "$")
    i += 1
plt.title(r"$\mathrm{Confocal\_FISH}$")
plt.ylabel(r"$\text{PCC}$")
#plt.ylabel(r"$\mathrm{PCC}(\mathbf{X}, \hat{\mathbf{X}})$")
plt.xlabel(r"$w$")
plt.legend()
#plt.show()
### barb_GF_optimal_tau
#plt.title(r"$\mathbf{X}=\text{Barb}$")
plt.savefig("Confocal_FISH__Wiener_scipy__optimal_window.pdf", bbox_inches='tight')

In [ ]:
plt.xlabel(r"$\sigma$")
plt.ylabel(r"$w^*$")
plt.plot(std_devs, optimal_sizes)

In [ ]:
optimal_sizes = np.array(optimal_sizes)
np.save("Confocal_FISH__Wiener_scipy_optimal_sizes.npy", optimal_sizes)